<a href="https://colab.research.google.com/github/asim5800/Deep-learning-practicals/blob/main/Boltzmann_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Boltzmann Machine

***We're gonna make the recommended system that predicts a binary outcome yes or no with our Boltzmann machines.data set is available on this website [link text](https://grouplens.org/datasets/movielens/).So this data set was created by GroupLens research.***

##Downloading the dataset

###ML-100K

In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-09-17 04:56:57--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  22.5MB/s    in 0.2s    

2021-09-17 04:56:57 (22.5 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2021-09-17 04:56:57--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  28.2MB/s    in 0.2s    

2021-09-17 04:56:58 (28.2 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [4]:

# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [5]:

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3883 non-null   int64 
 1   1       3883 non-null   object
 2   2       3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [6]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6040 non-null   int64 
 1   1       6040 non-null   object
 2   2       6040 non-null   int64 
 3   3       6040 non-null   int64 
 4   4       6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [8]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   0       1000209 non-null  int64
 1   1       1000209 non-null  int64
 2   2       1000209 non-null  int64
 3   3       1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [10]:
ratings.head()#user,moviesIDs,ratings,timestamp(dont care)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Preparing the training set and the test set


In [11]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')#(u1 base,u1 test we have 5 train test split like like this)

test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')


In [12]:



training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79999 entries, 0 to 79998
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   1          79999 non-null  int64
 1   1.1        79999 non-null  int64
 2   5          79999 non-null  int64
 3   874965758  79999 non-null  int64
dtypes: int64(4)
memory usage: 2.4 MB


In [13]:

test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   1          19999 non-null  int64
 1   6          19999 non-null  int64
 2   5          19999 non-null  int64
 3   887431973  19999 non-null  int64
dtypes: int64(4)
memory usage: 625.1 KB


In [14]:
training_set.head(5)

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


***So here for example, at the fourth index,that is the fifth observation because it started at zero well, that corresponds to the user number one that's rated the movie number seven, and gave it four stars.***

In [15]:
#converting dataframe into array as pytorch works with array 
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


We're gonna get the total number of users and movies.
We're gonna make two matrices,one for the training set and one for the test set.

These matrices will have the same number of users and the same number of movies, so they will have the same number of lines and the same number of columns.

And in these two matrices, each cell of indexes U, I, where U is the user and I is the movie, each cell U, I will get the rating of the movie I, by the user U.

And if, this user U didn't rate the movie I,we'll put a zero.

In [16]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))#max user id could be in train or test set so we check max out of these two split
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [17]:
#getting user and movies count in dataset
print("Total number of users are",nb_users)
print("Total number of movies are",nb_movies)

Total number of users are 943
Total number of movies are 1682


## Converting the data into an array with users in rows and movies in columns


This whole list of lists will be a list of 943 lists because we have 943 users, and each of these 943 lists will be a list of 1,682 elements because we have 1,682 movies.Alright so now let's create this list of lists.

In [18]:

#structuring our data for input in our BOLTZMANN MACHINE  943X1682 rowsxcolumns
def convert(data):
  new_data = [] #initilaizing list
  for id_users in range(1, nb_users + 1): #as upper bound in range is excluded as we want 943 not 943
    id_movies = data[:, 1] [data[:, 0] == id_users] #take all movie ids rated by each user respectively
    id_ratings = data[:, 2] [data[:, 0] == id_users] #take all ratings rated by each user respectively
    ratings = np.zeros(nb_movies)  #filling all rating by zero
    ratings[id_movies - 1] = id_ratings # if user watched and rated movie then subtract it and insert that into respective cell
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [19]:
len(training_set),len(test_set)

(943, 943)

In [20]:
for items in test_set:
    print(items)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

## Converting the data into Torch tensors


Tensors are simply arrays that contain elements of a single data type.
So a tensor is a multi-dimensional matrix but instead of being a NumPy array, this is a PyTorch array.

In fact, we could build a neural network with NumPy arrays,but that would be much less efficient

For autoencoders PyTorch is preferred.

In [21]:
#converting our list of list into torch tensors
training_set = torch.FloatTensor(training_set) #torch tensor are multidimensional arrays
test_set = torch.FloatTensor(test_set)

In [22]:
training_set #users in rows and movies in columns 

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [23]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


Zero, if the user didn't like the movie and one, if the user liked the movie.
But, why do we also need the inputs to have the binary format zero or one?

Well, that's simply because,the RBM will take the input vector and inside this input vector, it will predict the ratings for the moviesthat were not originally rated by the user.

But since these predicted ratings are computed originally, from the existing ratings of the input vector,well, then the predicted ratings in the output must have the same format as the existing ratings in the input.

Original zeroes in tensor(input) now have different meaning =dislike. 


In [24]:
training_set[training_set == 0] = -1 #earlier 0 means no rating but now -1 will represent no rating given by user(didn't watch)
training_set[training_set == 1] = 0  
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1   # liked only when rating>=3 
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [25]:
training_set

tensor([[-1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

## Creating the architecture of the Neural Network


 ***nv= no.of visible nodes (USER RATINGS), nh= no. of hidden nodes***

***x= visible node , y= hidden node*** 

In [26]:
class RBM():
  def __init__(self, nv, nh): #compulsory function in every class 
    self.W = torch.randn(nh, nv) #initializing weights randomly according to normal distribution tensor of size nh*nv
    self.a = torch.randn(1, nh)  #bias for p(hidden nodes| visible nodes) 
    self.b = torch.randn(1, nv)  #bias for p(v| h)
  
  def sample_h(self, x):    #sample the probabilities of the hidden nodes given the visible notes.
    wx = torch.mm(x, self.W.t()) #make product of x(visible nodes) & weights
    activation = wx + self.a.expand_as(wx) # wx+a same as every DL model (weight x input +bias) 
    p_h_given_v = torch.sigmoid(activation) # hidden node activated given value of visible node
    return p_h_given_v, torch.bernoulli(p_h_given_v)  #bernoulli RBM
  
  def sample_v(self, y):    #sample the probabilities of the visible notes given the hidden notes.
    wy = torch.mm(y, self.W) #torch tensor of hidden nodes multiply by torch tensor of weights
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  
   #contrastive divergence
  def train(self, v0, vk, ph0, phk): 
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t() #update weights and all biases 
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])  #visible nodes 1682 as number of movies 
nh = 100   # number of feature we want to detect by RBM model (genre,actor,oscar etc)
batch_size = 100 #update gradient after 100 observation
rbm = RBM(nv, nh)

#v0= input vector containing the ratings of all the movies by one user.
#vk= visible nodes obtained after K samplings.
#ph0= vector of probabilities that at the first iteration the hidden nodes equal one given the values of V zero
#phk= probabilities of the hidden nodes after K sampling given the values of the visible nodes, vk.

For example, let's say that we have a user that likes only dramatic movies.

Well, if there is a hidden node that detected a specific feature corresponding to that drama genre of the movies, well for this user who has high ratings, that is one ratings for all the movies that are drama movies.

The probability of that node that is specific to this drama, feature genre, given the visible node of that user who has all the nodes of the drama movies is equal to one.

This probability will be very high because v equals one for the drama movies and h corresponds to the drama movie genre.

So p (h given v) will be very high.
We have a vector of 1,682 probabilities,one probability for each of the visible nodes,

## Training the RBM


In [27]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0  #prediction real value difference
  s = 0.  # counter to normalise train loss incremented after each epoch    
  for id_user in range(0, nb_users - batch_size, batch_size):#taking user each batch one 1st 0-99
    vk = training_set[id_user : id_user + batch_size] #at start vk is our rating batch only taking 100 users batch
    v0 = training_set[id_user : id_user + batch_size] # ratings of the movies already rated by 100 users in this batch. 
    ph0,_ = rbm.sample_h(v0)  #initial probabilities p(h|v)= 1
    for k in range(10):   #k steps of contrastive divergence
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0] #freeze visible nodes having-1 as rating in cells
    phk,_ = rbm.sample_h(vk)  
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))  #MAE metric
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


#vk is our visible nodes that are being updated during the k steps of the random walk.
#v0 is our target

epoch: 1 loss: tensor(0.3441)
epoch: 2 loss: tensor(0.2468)
epoch: 3 loss: tensor(0.2500)
epoch: 4 loss: tensor(0.2474)
epoch: 5 loss: tensor(0.2463)
epoch: 6 loss: tensor(0.2495)
epoch: 7 loss: tensor(0.2483)
epoch: 8 loss: tensor(0.2499)
epoch: 9 loss: tensor(0.2441)
epoch: 10 loss: tensor(0.2456)


***Train loss of 0.25. IN Training set we're predicting 3 correct out of 4***

## Testing the RBM


In [28]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2468)


***Test loss of 0.25. In Testing set we're predicting 3 correct out of 4***